# Demo: Fast Geometric Projections for Local Robustness Certification

In [1]:
# Set the backend to TensorFlow.
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Prevent TensorFlow from its default behavior of eating all our RAM.
from keras import backend as K
from tensorflow import ConfigProto, GPUOptions, Session

Session()

K.set_session(
  Session(config=ConfigProto(
    gpu_options=GPUOptions(allow_growth=True))))

Using TensorFlow backend.


In [2]:
from checkable_models import SimpleSigmoidFfn, SimpleSoftmaxFfn
from datasets import Data
from robustness_certification import check

from time import time
from numpy.random import randint, seed

## Example: Checking Fashion-MNIST for 0.25-Robustness

In [3]:
# Load the model.
model = SimpleSoftmaxFfn(28*28, (20,)*3, 10).compile_backprop()
model.load_weights('models/fmnist20.20.20_weights.h5')

# Load the data.
fmnist_data = Data.mnist(fashion=True)

In [4]:
seed(4444)

# Run on 10 examples.
for i in range(10):
  print('\nCHECKING POINT {} / {}\n'.format(i+1, 10))
  
  start_time = time()
  
  result, regions_checked = check(
    model, 
    fmnist_data.test[randint(len(fmnist_data.test))],
    0.25,
    timeout=60,
    batch_size=16,
    keepgoing=True,
    return_num_visited=True,
    recap=True)
    
  end_time = time()
  
  print(
    '\n> {} - checked {} region(s) in {:.3f}s'
    .format(result, regions_checked, end_time - start_time))


CHECKING POINT 1 / 10

Started with 0 constraints to check.
No more constraints to check.
Checked 1 regions.

> ROBUST - checked 1 region(s) in 0.021s

CHECKING POINT 2 / 10

Started with 3 constraints to check.
No more constraints to check.
Checked 16 regions.

> ROBUST - checked 16 region(s) in 0.045s

CHECKING POINT 3 / 10

Started with 3 constraints to check.
Found decision boundary within epsilon radius.
Checked 1 regions.
Found a true adversarial example at distance 0.0885.

> NOT_ROBUST - checked 1 region(s) in 0.009s

CHECKING POINT 4 / 10

Started with 4 constraints to check.
No more constraints to check.
Checked 16 regions.

> ROBUST - checked 16 region(s) in 0.035s

CHECKING POINT 5 / 10

Started with 9 constraints to check.
No more constraints to check.
Checked 512 regions.

> ROBUST - checked 512 region(s) in 0.296s

CHECKING POINT 6 / 10

Started with 2 constraints to check.
No more constraints to check.
Checked 4 regions.

> ROBUST - checked 4 region(s) in 0.020s

CHECK

## Example: Checking the Lower Bound for a Larger Network

In [5]:
# Load the model.
model_big = SimpleSoftmaxFfn(28*28, (500,)*5, 10).compile_backprop()
model_big.load_weights('models/fmnist500.500.500.500.500_weights.h5')

In [6]:
seed(4444)

# Run on 5 examples (takes about 5 minutes).
for i in range(5):
  print('\nCHECKING POINT {} / {}\n'.format(i+1, 5))
    
  result, lower_bound, regions_checked = check(
    model_big, 
    fmnist_data.test[randint(len(fmnist_data.test))],
    1.,
    timeout=60,
    lowerbound=True,
    return_num_visited=True,
    recap=True)
      
  print(
    '\n> {} - proven robust up to {:.3f} after checking {} region(s)'
    .format(result, lower_bound, regions_checked))


CHECKING POINT 1 / 5

Started with 34 constraints to check.
0.027656024
0.034492105
0.058350805
0.08538272
0.1728914
0.17398275
0.19573721
0.23451981
0.24207468
Timed out.
Proven roubst up to epsilon = 0.24207468330860138

> TIMED_OUT - proven robust up to 0.242 after checking 312 region(s)

CHECKING POINT 2 / 5

Started with 146 constraints to check.
0.0012331223
0.06780169
0.076615594
0.08204681
0.103015386
0.10330306
0.124097265
Timed out.
Proven roubst up to epsilon = 0.12409726530313492

> TIMED_OUT - proven robust up to 0.124 after checking 311 region(s)

CHECKING POINT 3 / 5

Started with 38 constraints to check.
0.0037150686
0.02534551
0.06759798
0.076444365
0.08120409
0.08267096
0.09746346
0.10561491
0.12825698
Timed out.
Proven roubst up to epsilon = 0.12825697660446167

> TIMED_OUT - proven robust up to 0.128 after checking 312 region(s)

CHECKING POINT 4 / 5

Started with 44 constraints to check.
0.0017394
0.027268996
0.047306836
0.047532093
0.10444308
0.12879945
0.1460061